# **Deforum Google Colab**
Deforum for Google Colab. Generate animations with diffusion models. Features include: text-to-video, image-to-video, video-to-video, and keyframe camera motions. Supports SD1.5, SDXL, and SD3. Built on top of diffusers. Notebook by [Huemin](https://www.huemin.art)

In [1]:
#@markdown **SETUP ENVIRONMENT**
import subprocess, os, sys
sub_p_res = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total,memory.free', '--format=csv,noheader'], stdout=subprocess.PIPE).stdout.decode('utf-8')
print(f"{sub_p_res[:-1]}")

models_path = "/content/models"
output_path = "/content/output"

# Mount Google Drive
mount_google_drive = True
force_remount = False

if mount_google_drive:
    from google.colab import drive
    try:
        drive_path = "/content/drive"
        drive.mount(drive_path,force_remount=force_remount)
        models_path_gdrive = "/content/drive/MyDrive/AI/models"
        output_path_gdrive = "/content/drive/MyDrive/AI/outputs"
        models_path = models_path_gdrive
        output_path = output_path_gdrive
    except:
        print("...error mounting drive or with drive path variables")
        print("...reverting to default path variables")

import os
os.makedirs(models_path, exist_ok=True)
os.makedirs(output_path, exist_ok=True)

print(f"models_path: {models_path}")
print(f"output_path: {output_path}")

# install stuff
!git clone https://github.com/huemin-art/deforum-google-colab.git
!pip install -q diffusers transformers accelerate

Tesla T4, 15360 MiB, 15101 MiB
Mounted at /content/drive
models_path: /content/drive/MyDrive/AI/models
output_path: /content/drive/MyDrive/AI/outputs
Cloning into 'deforum-google-colab'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 19 (delta 6), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (19/19), 14.99 KiB | 3.75 MiB/s, done.
Resolving deltas: 100% (6/6), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 63.0 MB/s eta 0:00:00


In [10]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGr

In [11]:
from re import T
import torch
from pydantic import BaseModel, computed_field
from diffusers import AutoPipelineForText2Image, AutoPipelineForImage2Image
from IPython.display import display, clear_output, Video
from tqdm.notebook import trange

sys.path.append(os.path.join(os.getcwd(), "deforum-google-colab"))

from helpers import load_image, create_animation_from_pil_images

#@markdown **RUN MODE**
mode = "image" #@param ["image", "animation"]

model_map = {
    "sd3-medium": {
        "model": "stabilityai/stable-diffusion-3-medium-diffusers",
        "device": "cuda",
        "variant": None,
        "safetensors": False,
        "torch_dtype": "float16"
    },
    "sdxl-protovisionxl": {
        "model": "stablediffusionapi/protovision-xl-high-fidel",
        "device": "cuda",
        "variant": None,
        "safetensors": False,
        "torch_dtype": "float16"
    },
    "sdxl-base": {
        "model": "stabilityai/stable-diffusion-xl-base-1.0",
        "device": "cuda",
        "variant": "fp16",
        "safetensors": True,
        "torch_dtype": "float16"
    }
}

#@markdown **MODEL SETTINGS**
class DeforumConfig(BaseModel):
    name: str = "sd3-medium" #@param ["sdxl-protovisionxl", "sdxl-base-sdxl","sd3-medium"]
    model: str = ""
    device: str = "cuda"
    variant: str = None
    safetensors: bool = True
    torch_dtype: str = "float16"

    def __init__(self, **data):
        super().__init__(**data)
        self.update_from_model_map()

    def update_from_model_map(self):
        if self.name in model_map:
            config = model_map[self.name]
            for key, value in config.items():
                setattr(self, key, value)

#@markdown **IMAGE SETTINGS**
class DeforumArgs(BaseModel):
    prompt: str = "cat sushi" #@param {type:"string"}
    negative_prompt: str = "" #@param {type:"string"}
    width: str = 768 #@param [256, 320, 384, 448, 512, 576, 640, 704, 768, 832, 896, 960, 1024, 1088, 1152, 1216, 1280, 1344, 1408, 1472, 1536]
    height: int = 768 #@param [256, 320, 384, 448, 512, 576, 640, 704, 768, 832, 896, 960, 1024, 1088, 1152, 1216, 1280, 1344, 1408, 1472, 1536]
    num_inference_steps: int = 20 #@param {type:"integer"}
    guidance_scale: float = 4.0 #@param {type:"number"}
    strength: float = 0.5 #@param {type:"number"}
    init_image: str = "" #@param {type:"string"}

#@markdown **ANIMATION SETTINGS**
class DeforumAnimationArgs(BaseModel):
    duration_seconds: int = 30 #@param {type:"integer"}
    frames_per_second: int = 12 #@param {type:"integer"}

    @computed_field
    def max_frames(self) -> int:
        return self.duration_seconds * self.frames_per_second

def get_txt2img(config):
    if config.name == "sd3-medium":
        from diffusers import StableDiffusion3Pipeline, StableDiffusion3Img2ImgPipeline
        txt2img = StableDiffusion3Pipeline.from_pretrained(
            config.model,
            torch_dtype=getattr(torch, config.torch_dtype)
        ).to(config.device)
        img2img = StableDiffusion3Img2ImgPipeline.from_pipe(txt2img)
    else:
        txt2img = AutoPipelineForText2Image.from_pretrained(
            config.model,
            torch_dtype=getattr(torch, config.torch_dtype),
            variant=config.variant,
            use_safetensors=config.safetensors
        ).to(config.device)
        img2img = AutoPipelineForImage2Image.from_pipe(txt2img)

    return txt2img, img2img

class DeforumGenerator:
    def __init__(self, config: DeforumConfig):
        self.config = config
        self.txt2img, self.img2img = get_txt2img(config)

        # state
        self.init = None
        self.frames = []

    def generate_image(self, args: DeforumArgs):

        if self.init is None:
            self.init = load_image(args.init_image)

        if self.init is None:
            image = self.txt2img(
                prompt=args.prompt,
                negative_prompt=args.negative_prompt,
                width=args.width,
                height=args.height,
                num_inference_steps=args.num_inference_steps,
                guidance_scale=args.guidance_scale
            ).images[0]
        else:
            image = self.img2img(
                prompt=args.prompt,
                image=self.init,
                negative_prompt=args.negative_prompt,
                width=args.width,
                height=args.height,
                num_inference_steps=args.num_inference_steps,
                guidance_scale=args.guidance_scale,
                strength=args.strength
            ).images[0]

        image.save("output.png")
        clear_output(wait=True)
        display(image)

        return image


    def generate_animation(self, args: DeforumArgs, anim_args: DeforumAnimationArgs):

        # reset
        self.init = None
        self.frames = []

        # loop
        for i in trange(anim_args.max_frames):
            print(args)
            self.init = self.generate_image(args)
            self.frames.append(self.init)

        # video
        create_animation_from_pil_images(
            self.frames,
            "test.mp4",
            fps=anim_args.frames_per_second,
            loop=0,
            crf=17
        )

        Video("test.mp4", embed=True)


if __name__ == "__main__":
    if "deforum" not in globals():
        config = DeforumConfig()
        deforum = DeforumGenerator(config)

    args = DeforumArgs()
    anim_args = DeforumAnimationArgs()

    if mode == "image":
        print(args)
        deforum.generate_image(args)
    if mode == "animation":
        print(anim_args)
        deforum.generate_animation(args, anim_args)


Couldn't connect to the Hub:  (Request ID: Root=1-6680c6be-185269895ff0cf436aa109c4;d27dbab5-3c08-4085-9152-0aa93e90d0b6)

403 Forbidden: Please enable access to public gated repositories in your fine-grained token settings to view this repository..
Cannot access content at: https://huggingface.co/api/models/stabilityai/stable-diffusion-3-medium-diffusers.
If you are trying to create or update content,make sure you have a token with the `write` role..
Will try to load from local cache.


OSError: Cannot load model stabilityai/stable-diffusion-3-medium-diffusers: model is not cached locally and an error occurred while trying to fetch metadata from the Hub. Please check out the root cause in the stacktrace above.